### DANGEROUS TERRITORY:
This notebook can allocate a lot of disk space and needs some processing power (ideally available CUDA GPU)

# Reddit Climate Change - Modeling Sentiment & Emotion
Supervision: Prof. Dr. Jan Fabian Ehmke

Group members: Britz Luis, Huber Anja, Krause Felix Elias, Preda Yvonne-Nadine

Time: Summer term 2023 

Data: https://www.kaggle.com/datasets/pavellexyr/the-reddit-climate-change-dataset

In [1]:
from transformers import pipeline # Protobuf version <4 (e.g. 3.20.3) might be needed!
import pandas as pd
import numpy as np
import os
import torch

print(f"CUDA device found: {torch.cuda.get_device_name(torch.cuda.current_device())}")
if torch.cuda.is_available():
    device = torch.cuda.current_device()
    print("### \n WARNING: YOU WILL TRAIN ON DETECTED GPU \n###")
else:
    device = -1

CUDA device found: NVIDIA GeForce GTX 1060 6GB
### 
###


## Load data

In [3]:
# Load comments
df = pd.read_csv("data/comments_preprocessed.gzip", compression="gzip", header=0)#, skiprows=0, nrows=11000) # FIXME For now only small sample!
# TODO Maybe filter e.g. for year/range
df.head(3)

<ipython-input-3-a08cb6e9261b>:2: DtypeWarning: Columns (0,1,2,4,7,11,12) have mixed types. Specify dtype option on import or set low_memory=False.
  df = pd.read_csv("data/comments_preprocessed.gzip", compression="gzip", header=0)#, skiprows=0, nrows=11000) # FIXME For now only small sample!


,id,subreddit.name,subreddit.nsfw,created_utc,permalink,sentiment,score,created_date,created_day,created_month,created_year,created_time,body_clean
0,i79uz1c,oddlyterrifying,False,1.651658e+09,https://old.reddit.com/r/oddlyterrifying/comme...,-0.5574,3.0,2022-05-04,4.0,5.0,2022.0,09:58:26,Oh shit there's a new one out? Last one k watc...
1,hz51unj,technews,False,1.646280e+09,https://old.reddit.com/r/technews/comments/t53...,0.4588,1.0,2022-03-03,3.0,3.0,2022.0,03:54:08,"We’re never going to reopen those wells, its e..."
2,i3ic64d,worldnews,False,1.649177e+09,https://old.reddit.com/r/worldnews/comments/tw...,0.6249,1.0,2022-04-05,5.0,4.0,2022.0,16:36:35,Climate Change is the Great Filter.


In [7]:
df["id"] = df["id"].astype(str)
df["title"] = df["title"].astype(str)
# df["body"] = df["body"].astype(str)

In [9]:
df.dtypes

id                 object
subreddit.name     object
subreddit.nsfw     object
created_utc       float64
permalink          object
sentiment         float64
score             float64
created_date       object
created_day       float64
created_month     float64
created_year      float64
created_time       object
body_clean         object
dtype: object

In [10]:
df.shape

(1500255, 13)

In [16]:
df[df.isna().any(axis=1)]

,id,subreddit.name,subreddit.nsfw,created_utc,permalink,sentiment,score,created_date,created_day,created_month,created_year,created_time,body_clean
188548,h6kpauz,polls,False,1.627298e+09,https://old.reddit.com/r/polls/comments/orl7mj...,-0.3612,2.0,2021-07-26,26.0,7.0,2021.0,11:17:44,NaN
188549,Stop climate change and start WW because WW ...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
494626,- Targeted cuts to research on climate change,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
494627,- Openly calling for killing civilians in the ...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
518932,You *walk on* a recording of climate change *...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...
1376256,.html,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1385567,Being skeptical of this is as bad as denying t...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1394370,German school children still write letters to ...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1424093,Edit: a word,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [18]:
df.shape

(1500255, 13)

In [ ]:
# Year range: 2010-2022
year = 2010

In [110]:
# Extract sample text
df.body.iloc[10]

'https://www.google.com/amp/s/www.foxnews.com/opinion/climate-change-congress-must-act-cleaner-healthier-world.amp\n\nThat sure is some far left propaganda, fox news...'

Faulty rows of old preprocessed_comments file: 

368464 - 393215:    "i38o4bg	climatechange	False	1649003948	"

393217 - 425983:    "0.0	[AR IPCC Chapter : Understanding and Attribu...	NaN"



## HF Transformers Models

Be aware: Models rather large, first time to run might take some downloading time (~500MB per model; saved in sth like "C:\Users\Felix\.cache\huggingface\hub")

All HF sentiment models: https://huggingface.co/models?pipeline_tag=text-classification&sort=downloads&search=sentiment

All HF emotion detection models: https://huggingface.co/models?pipeline_tag=text-classification&sort=downloads&search=emotion

### Sentiment Model

https://huggingface.co/cardiffnlp/twitter-roberta-base-sentiment

ENCODING: 0 -> Negative; 1 -> Neutral; 2 -> Positive

In [ ]:
model = pipeline("sentiment-analysis", model="cardiffnlp/twitter-roberta-base-sentiment", device=device)

### Climate stance model

https://huggingface.co/cardiffnlp/twitter-roberta-base-stance-climate

Paper: https://aclanthology.org/S16-1003.pdf

"Climate Change is a Real Concern" -> favor/against/none

~0.014 sec/img

In [45]:
model = pipeline("sentiment-analysis", model="cardiffnlp/twitter-roberta-base-stance-climate", device=device)

### Climate sentiment model

https://huggingface.co/climatebert/distilroberta-base-climate-sentiment 

-> neutral, opportunity, risk

In [ ]:
model = pipeline("sentiment-analysis", model="climatebert/distilroberta-base-climate-sentiment", device=device)

### Large emotion model (28 states detected)

Model: https://huggingface.co/arpanghoshal/EmoRoBERTa

Labels detected: 'remorse', 'disappointment', 'sadness', 'gratitude', 'realization', 'disapproval', 'neutral', 'approval', 'embarrassment', 'caring', 'curiosity', 'confusion', 'annoyance', 'joy', 'optimism', 'relief', 'excitement', 'admiration', 'love', 'disgust', 'grief', 'amusement', 'anger', 'surprise', 'pride', 'nervousness', 'fear', 'desire'

In [40]:
model = pipeline('sentiment-analysis', model='arpanghoshal/EmoRoBERTa', top_k=3, device=device) # top_k=None lists all labels

All model checkpoint layers were used when initializing TFRobertaForSequenceClassification.

All the layers of TFRobertaForSequenceClassification were initialized from the model checkpoint at arpanghoshal/EmoRoBERTa.
If your task is similar to the task the model of the checkpoint was trained on, you can already use TFRobertaForSequenceClassification for predictions without further training.


### Small emotion model (7 states)

Model: https://huggingface.co/j-hartmann/emotion-english-distilroberta-base

Labels detected: 'surprise', 'neutral', 'fear', 'anger', 'joy', 'disgust', 'sadness'

(~11h to label all comments)

In [107]:
model = pipeline("text-classification", model="j-hartmann/emotion-english-distilroberta-base", top_k=1, device=device) # top_k=None lists all labels

## Test model functionality

In [46]:
sample_prediction = model("Climate change is just a hoax! Why is everyone so upset?!?!")
sample_prediction

[{'label': 'favor', 'score': 0.9345217347145081}]

In [ ]:
# Find labels included
#labels = [i["label"] for i in sample_prediction]
#print(labels)

## Apply model

In [58]:
# Reduce text to character limit if there is one
def reduce_text(text, max_length=np.inf):
    if len(text) > max_length:
        text = text[0:max_length]
    return text

In [48]:
# Reduce texts if necessary
#df["body_clean"] = df["body_clean"].apply(reduce_text, args=(512, )) # e.g. for small sentiment model

In [49]:
# CAUTION: THIS WILL RUN INFERENCE AND CAN THUS TAKE SOME TIME
df["label"] = model(list(df["body_clean"]))

In [50]:
# Extract (top) label
df["top_label"] = [i["label"] for i in df["label"]]

In [51]:
# Print all top labels found
df.top_label.unique()

array(['favor', 'none'], dtype=object)

In [57]:
# Get sample texts for certain label
df[df.top_label == "none"].body_clean.iloc[3]

'This has nothing to do with  . Donnie the orange says   is a conspiracy theory.'

## Save results

In [ ]:
comment_save_path = f"data/comments_inference.gzip"

if not os.path.isfile(comment_save_path):
    df.to_csv(comment_save_path, index=False, compression="gzip")
    print("File saved!")
else: 
    print("Warning file already exists!")